In [0]:
dbutils.widgets.removeAll()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
dbutils.widgets.text("container", "raw-jptq")
dbutils.widgets.text("catalogo", "catalog_jptq")
dbutils.widgets.text("esquema", "bronze_jptq")
dbutils.widgets.text("datalake", "adlsjptq0126")
dbutils.widgets.text("file", "Peliculas")
dbutils.widgets.text("name_file", "MoreInfo.csv")

In [0]:
container = dbutils.widgets.get("container")
catalogo = dbutils.widgets.get("catalogo")
esquema = dbutils.widgets.get("esquema")
datalake = dbutils.widgets.get("datalake")
file = dbutils.widgets.get("file")
name_file = dbutils.widgets.get("name_file")
ruta = f"abfss://{container}@{datalake}.dfs.core.windows.net/{file}/{name_file}"

In [0]:
df_more_info_movies = spark.read.option('header', True)\
                        .option('inferSchema', True)\
                        .csv(ruta)

In [0]:
more_info_movies_schema = StructType(fields=[StructField("id", IntegerType(), False),
                                     StructField("runtime", StringType(), True),
                                     StructField("budget", StringType(), True),
                                     StructField("revenue", StringType(), True),
                                     StructField("film_id", IntegerType(), True)
])

In [0]:
df_more_info_movies_final = spark.read\
.option('header', True)\
.schema(more_info_movies_schema)\
.csv(ruta)

In [0]:
more_info_movies_selected_df = df_more_info_movies_final.select(col("id"), 
                                                col("runtime"), 
                                                col("budget"), 
                                                col("revenue"), 
                                                col("film_id"))

In [0]:
more_info_movies_renamed_df = more_info_movies_selected_df.withColumnRenamed("id", "ID_PELICULA") \
                                            .withColumnRenamed("runtime", "DES_DURACION_PELICULA") \
                                            .withColumnRenamed("budget", "VAL_PRESUPUESTO") \
                                            .withColumnRenamed("revenue", "VAL_INGRESOS") \
                                            .withColumnRenamed("film_id", "PELICULA_ID") 

In [0]:
more_info_movies_final_df = more_info_movies_renamed_df.withColumn("FEC_CARGA", current_timestamp())

In [0]:
more_info_movies_final_df.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema}.TBL_MAS_INFO_PELICULAS")

In [0]:
%sql
SELECT
*
FROM
catalog_jptq.bronze_jptq.tbl_mas_info_peliculas